# **Análise Exploratória de Dados (Loggi)**

## 1\. Contexto
Esse Projeto faz parte da formação "Analista de Dados" da EBAC e foi realizado em parceria com a empresa Loggi. Comessaremos com os dados de entrega, fornecidos pela empresa, referentes a cidade de brasília, em formato JSON. Iremos estruturar esses dados em um formato tabular, manipular-los, enrriquecer-los para, a partir deles, gerar visualisações e insigths.

In [ ]:
!pip3 install geopandas

In [ ]:
# Importações

import json

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# Download dos dados brutos em formato JSON

!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json

In [ ]:
# Download dos dados de localização convertidos de coordenadas para endereços (entrega por entrega)

!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv" -O deliveries-geodata.csv

In [ ]:
# Download do mapa do distrito federal através do site oficial do IBGE

!wget -q "https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip" -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

In [ ]:
# Estruturando o dado JSON em um formato tabular

with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

deliveries_df = pd.DataFrame(data)
origin_df = pd.json_normalize(deliveries_df["origin"])

deliveries_df = pd.merge(left=deliveries_df, right=origin_df, how='inner', left_index=True, right_index=True)
deliveries_df = deliveries_df.drop("origin", axis=1)
deliveries_df = deliveries_df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
deliveries_df.rename(columns={"lng": "origin_lng", "lat": "origin_lat"}, inplace=True)

deliveries_exploded_df = deliveries_df[["deliveries"]].explode("deliveries")
deliveries_normalized_df = pd.concat([
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["size"])).rename(columns={"deliveries": "delivery_size"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lng"])).rename(columns={"deliveries": "delivery_lng"}),
  pd.DataFrame(deliveries_exploded_df["deliveries"].apply(lambda record: record["point"]["lat"])).rename(columns={"deliveries": "delivery_lat"}),
  ], axis= 1)

deliveries_df = deliveries_df.drop("deliveries", axis=1)
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
deliveries_df.reset_index(inplace=True, drop=True)

In [ ]:
deliveries_df.head()

In [ ]:
# Fazendo a geocodificação reversa dos HUBS de distribuição

hubs = deliveries_df[["region", "origin_lng", "origin_lat"]]
hubs = hubs.drop_duplicates().sort_values(by="region").reset_index(drop=True)

geolocator = Nominatim(user_agent="ebac_geocoder")
location = geolocator.reverse("-15.657013854445248, -47.802664728268745")
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

hubs["coordinates"] = hubs["origin_lat"].astype(str)  + ", " + hubs["origin_lng"].astype(str)
hubs["geodata"] = hubs["coordinates"].apply(geocoder)

hubs.head()

In [ ]:
# Aprimorando o Data Frame de entrgas

hub_geodata_df = pd.json_normalize(hubs["geodata"].apply(lambda data: data.raw))
hub_geodata_df = hub_geodata_df[["address.town", "address.suburb", "address.city"]]
hub_geodata_df.rename(columns={"address.town": "hub_town", "address.suburb": "hub_suburb", "address.city": "hub_city"}, inplace=True)
hub_geodata_df["hub_city"] = np.where(hub_geodata_df["hub_city"].notna(), hub_geodata_df["hub_city"], hub_geodata_df["hub_town"])
hub_geodata_df["hub_suburb"] = np.where(hub_geodata_df["hub_suburb"].notna(), hub_geodata_df["hub_suburb"], hub_geodata_df["hub_city"])
hub_geodata_df = hub_geodata_df.drop("hub_town", axis=1)
hubs = pd.merge(left=hubs, right=hub_geodata_df, left_index=True, right_index=True)
hubs = hubs[["region", "hub_suburb", "hub_city"]]

deliveries_df = pd.merge(left=deliveries_df, right=hubs, how="inner", on="region")
deliveries_df = deliveries_df[["name", "region", "origin_lng", "origin_lat", "hub_city", "hub_suburb", "vehicle_capacity", "delivery_size", "delivery_lng", "delivery_lat"]]
deliveries_df.head()

In [ ]:
# Uma vez que há mais de seicentas mil consultas a serem feitas a respeito do local da entrega, seria inviável converter pelo código.
# Por isso será um arquivo csv com as coordenadas previamente convertidas para endereços.

deliveries_geodata_df = pd.read_csv('deliveries-geodata.csv')
deliveries_geodata_df.head(n=6)

In [ ]:
deliveries_df = pd.merge(left=deliveries_df, right=deliveries_geodata_df[['delivery_city', 'delivery_suburb']],
                         how='inner', left_index=True, right_index=True)
deliveries_df.head(n=7)

## 2\. Verificando a qualidade dos dados
Uma vez que os dados já foram tratados e enrriquecidos, nos resta verificar a consistência do schema e a presença de valores faltantes. Dessa forma será possível dimensionar a qualidade dos dados obtidos.


In [ ]:
# Observe que o Data Frame criamos possui as dimenções e tipo de dado esperado em cada coluna
deliveries_df.info()

In [ ]:
# Como foi verificado no código anterior, possuímos valores faltantes em duas colunas.
# Também podemos verificar isso com o seguint3e comando.
deliveries_df.isna().any()

In [ ]:
# Verifica-se que são os valores obtidos com a geocodificação reversa das entregas
# Vamos agora dimensionar o problema.

100 * (deliveries_df['delivery_city'].isna().sum() / len(deliveries_df))

In [ ]:
100 * (deliveries_df['delivery_suburb'].isna().sum() / len(deliveries_df))

In [ ]:
# Analisando os valores que mais se repetem e o seu percentual em relação ao total

prop_df = deliveries_df[["delivery_city"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

In [ ]:
# Tanto na célula de código anterior, como na atual vemos que alguns dos valores não nos interessam,
# uma vez que contem o valor "Brasília" e só estamos analisando dados do DF

prop_df = deliveries_df[["delivery_suburb"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(10)

## 3\. Criando a visualização dos dados
Aqui, vamos alocar marcar no mapa do distrito federal onde que é cada entrega diferenciando-os por seu hub de origem.

In [ ]:
mapa = geopandas.read_file('distrito-federal.shp')
mapa = mapa.loc[[0]]
mapa.head()

In [ ]:
# Primeiramente iremos criar um data frame contendo as informações necessárias para criar um mapa com is Hubs

hubs_df = deliveries_df[['region', 'origin_lng', 'origin_lat']].drop_duplicates().reset_index(drop=True)
geo_hub_df = geopandas.GeoDataFrame(hubs_df, geometry=geopandas.points_from_xy(hubs_df['origin_lng'], hubs_df['origin_lat']))
geo_hub_df.head()

In [ ]:
# Agora faremos os mesmos com as entregas

geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df,
                                           geometry=geopandas.points_from_xy(deliveries_df['delivery_lng'],
                                                                                            deliveries_df['delivery_lat']))
geo_deliveries_df.head()

In [ ]:
# Criando o mapa em si

fig, ax = plt.subplots(figsize = (50/2.54, 50/2.54))
mapa.plot(ax=ax, alpha=0.4, color='lightgrey')

geo_deliveries_df.query('region == "df-0"').plot(ax=ax, markersize=1, color='red', label='df-0')
geo_deliveries_df.query('region == "df-1"').plot(ax=ax, markersize=1, color='blue', label='df-1')
geo_deliveries_df.query('region == "df-2"').plot(ax=ax, markersize=1, color='seagreen', label='df-2')

geo_hub_df.plot(ax=ax, markersize=30, marker='x', color='black', label='hub')

plt.title('Entregas no DF por Região', fontdict={'fontsize':16})
lgnd = plt.legend(prop={'size': 15})

for handle in lgnd.legendHandles:
  handle.set_sizes([50])

## 4\. Storytelling

As entregas partem, em sua maioria dos Hubs df-1 e df-2, sendo muito menor a quantidade de entregas oriundas do Hub df-0. Contudo, o Hub df-0 possui um raio de entregas muito maior que os demais. Por sua vez, o Hub df-1, embora concentre o maior número de entregas, necessita cobrir um raio de entregas menor.
Assim sendo, se a capacidade de veículos é igual para todos os hubs, seria interessante fazer uma análise se essa distribuição é eficiente em dois sentidos: O primeiro seria a quantidade de frota ociosa no hub df-0, por ter um menor fluxo de entregas; O outro sentido seria averiguar se a frota do hub df-0 passa mais tempo do que os demais hubs realizando deslocamentos, o que implicaria em um custo maior de entrega. Uma possível solução seria pensar uma frota com características diferentes para cada Hub, tendo o hub df-1 veiculos maiores, que possibilitem realizar uma viagem maior enquanto fazem entregas em múltiplos destinos, o hub df-0 veículos menores e mais econômicos que não gastem tanto tempo e combustível em viagens que visam realizar entregas isoladas. Por conseguinte, o hub df-2 teria características hibridas.

Outro fato que é digno de nota foi em relação a coleta de endereços, no qual tivemos um pequeno problema com o schema de algumas regiões. Um dos motivos que pode ter ocasionado essa divergência se deve ao fato do Distrito Federal, por força do art. 32 da CRFB não poder ser dividido em municípios. Contudo, Isso não impede que o Destrito Federal seja dividido em 33 regiões administrativa, sendo uma delas, Brasília. Nesse sentido, talvez fosse mais interessante registrar em qual região administrativa é feita a entrega para que seja possível extrair métricas mais interessantes.